# 多交易所套利策略

**跨交易所资金费率与基差套利实战指南**

---

## 目录

### 第一部分：基础概念
1. 跨交易所套利简介
2. 套利类型概述
3. 主流交易所对比

### 第二部分：资金费率跨所套利
4. 不同交易所资金费率差异
5. 跨所套利策略设计
6. 成本分析
7. 实时机会扫描器

### 第三部分：基差跨所套利
8. 跨交易所基差分析
9. 基差套利策略实现

### 第四部分：实战系统
10. 多交易所 API 连接
11. 套利信号监控
12. 风险管理
13. 回测与绩效分析

---

# 第一部分：基础概念

---

## 1. 跨交易所套利简介

### 1.1 什么是跨交易所套利？

**跨交易所套利** 是指利用同一资产在不同交易所之间的价格差异或费率差异获取无风险或低风险收益的策略。

### 1.2 套利的基本原理

```
┌─────────────────────────────────────────────────────────────────┐
│                      跨交易所套利流程                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│     交易所 A                          交易所 B                  │
│   ┌───────────┐                    ┌───────────┐               │
│   │ 资金费率   │                    │ 资金费率   │               │
│   │ +0.05%    │                    │ +0.01%    │               │
│   └─────┬─────┘                    └─────┬─────┘               │
│         │                                │                      │
│         └────────────┬───────────────────┘                      │
│                      │                                          │
│                      ▼                                          │
│              ┌───────────────┐                                 │
│              │  费率差 0.04%  │                                 │
│              │  套利机会！    │                                 │
│              └───────────────┘                                 │
│                      │                                          │
│                      ▼                                          │
│         ┌────────────────────────────┐                         │
│         │ A 交易所做空 + B 交易所做多 │                         │
│         │ 净收益 = 费率差 - 成本      │                         │
│         └────────────────────────────┘                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 1.3 套利的优势与风险

| 优势 | 风险 |
|------|------|
| 市场中性，不受方向影响 | 执行风险（延迟、滑点） |
| 收益相对稳定 | 资金效率较低 |
| 可量化分析 | 交易所风险 |
| 自动化程度高 | 费率反转风险 |

## 2. 套利类型概述

### 2.1 资金费率套利

利用不同交易所之间资金费率的差异：
- **同向费率套利**：两边都是正费率，但一边更高
- **反向费率套利**：一边正费率，一边负费率

### 2.2 基差套利

利用同一资产在不同交易所的价格差：
- **正向基差套利**：低价买入，高价卖出
- **统计套利**：基于历史价差回归

### 2.3 三角套利

利用三个或更多交易对之间的价格不一致：
```
BTC/USDT → ETH/BTC → ETH/USDT → USDT
```

### 2.4 套利类型对比

| 类型 | 复杂度 | 收益潜力 | 风险 | 资金要求 |
|------|--------|----------|------|----------|
| 资金费率套利 | 低 | 中 | 低 | 中 |
| 基差套利 | 中 | 中 | 中 | 高 |
| 三角套利 | 高 | 低 | 低 | 高 |

## 3. 主流交易所对比

### 3.1 交易所特点对比

| 交易所 | 资金费率周期 | 费率上限 | 合约手续费(Maker/Taker) | API限制 |
|--------|-------------|----------|------------------------|--------|
| **Binance** | 8小时 | ±0.75% | 0.02%/0.04% | 1200次/分钟 |
| **OKX** | 8小时 | ±1.5% | 0.02%/0.05% | 300次/2秒 |
| **Bybit** | 8小时 | ±0.75% | 0.01%/0.06% | 120次/秒 |
| **dYdX** | 1小时 | 无上限 | 0.02%/0.05% | 按账户等级 |

### 3.2 各交易所优劣势

```
┌─────────────────────────────────────────────────────────────────┐
│                      交易所优劣势分析                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Binance                                                       │
│  ├── 优势：流动性最好、API稳定、币种最全                        │
│  └── 劣势：费率上限较低、部分地区受限                          │
│                                                                 │
│  OKX                                                           │
│  ├── 优势：费率上限高、资金费率波动大                          │
│  └── 劣势：API偶尔不稳定                                       │
│                                                                 │
│  Bybit                                                         │
│  ├── 优势：Maker费率低、API响应快                              │
│  └── 劣势：小币种流动性差                                      │
│                                                                 │
│  dYdX                                                          │
│  ├── 优势：去中心化、1小时结算周期                             │
│  └── 劣势：仅支持少数币种、Gas费用                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

# 第二部分：资金费率跨所套利

---

## 4. 不同交易所资金费率差异

In [ ]:
# 4.1 导入依赖

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import warnings
import time

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print('✅ 依赖库导入成功！')

In [ ]:
# 4.2 初始化多个交易所

def init_exchanges():
    """
    初始化多个交易所连接
    """
    exchanges = {}
    
    # Binance
    exchanges['binance'] = ccxt.binance({
        'enableRateLimit': True,
        'options': {'defaultType': 'swap'}
    })
    
    # OKX
    exchanges['okx'] = ccxt.okx({
        'enableRateLimit': True,
        'options': {'defaultType': 'swap'}
    })
    
    # Bybit
    exchanges['bybit'] = ccxt.bybit({
        'enableRateLimit': True,
        'options': {'defaultType': 'swap'}
    })
    
    # 加载市场
    for name, exchange in exchanges.items():
        try:
            exchange.load_markets()
            print(f'✅ {name}: 已连接，{len(exchange.markets)} 个市场')
        except Exception as e:
            print(f'❌ {name}: 连接失败 - {e}')
    
    return exchanges

exchanges = init_exchanges()

In [ ]:
# 4.3 获取各交易所资金费率

def get_funding_rates(exchanges: dict, symbol: str = 'BTC/USDT:USDT') -> dict:
    """
    获取各交易所当前资金费率
    """
    results = {}
    
    symbol_map = {
        'binance': symbol,
        'okx': symbol.replace(':USDT', ':USDT'),
        'bybit': symbol.replace('/USDT:USDT', 'USDT'),
    }
    
    for name, exchange in exchanges.items():
        try:
            sym = symbol_map.get(name, symbol)
            if sym in exchange.markets:
                funding = exchange.fetch_funding_rate(sym)
                results[name] = {
                    'funding_rate': funding.get('fundingRate', 0) * 100,
                    'next_funding_time': funding.get('fundingTimestamp'),
                    'mark_price': funding.get('markPrice', 0)
                }
        except Exception as e:
            print(f'{name}: {e}')
    
    return results

# 获取 BTC 资金费率
print('📊 获取各交易所 BTC 资金费率...\n')
btc_rates = get_funding_rates(exchanges)

print(f'{"交易所":<12} {"资金费率":<12} {"年化费率":<12}')
print('-' * 40)
for name, data in btc_rates.items():
    rate = data['funding_rate']
    annual = rate * 3 * 365
    print(f'{name:<12} {rate:>8.4f}%   {annual:>8.2f}%')

In [ ]:
# 4.4 多币种资金费率对比

def compare_funding_rates(exchanges: dict, symbols: list) -> pd.DataFrame:
    """
    对比多个币种在不同交易所的资金费率
    """
    data = []
    
    for symbol in symbols:
        rates = get_funding_rates(exchanges, symbol)
        row = {'symbol': symbol.split('/')[0]}
        for name, info in rates.items():
            row[name] = info['funding_rate']
        data.append(row)
        time.sleep(0.5)  # 避免频率限制
    
    df = pd.DataFrame(data)
    
    # 计算费率差
    if 'binance' in df.columns and 'okx' in df.columns:
        df['binance_okx_diff'] = df['binance'] - df['okx']
    if 'binance' in df.columns and 'bybit' in df.columns:
        df['binance_bybit_diff'] = df['binance'] - df['bybit']
    
    return df

# 对比主流币种
SYMBOLS = [
    'BTC/USDT:USDT',
    'ETH/USDT:USDT',
    'SOL/USDT:USDT',
    'XRP/USDT:USDT',
    'DOGE/USDT:USDT',
]

print('📊 获取多币种资金费率对比...')
df_rates = compare_funding_rates(exchanges, SYMBOLS)
print(df_rates.to_string(index=False))

## 5. 跨所套利策略设计

In [ ]:
# 5.1 套利机会计算器

class CrossExchangeArbitrage:
    """
    跨交易所套利计算器
    """
    def __init__(
        self,
        maker_fee: float = 0.0002,  # 0.02%
        taker_fee: float = 0.0005,  # 0.05%
        transfer_fee: float = 0.0001,  # 转账费用
        slippage: float = 0.0002  # 滑点
    ):
        self.maker_fee = maker_fee
        self.taker_fee = taker_fee
        self.transfer_fee = transfer_fee
        self.slippage = slippage
        
        # 单边成本
        self.single_side_cost = taker_fee + slippage
        # 双边总成本（开仓+平仓，两个交易所）
        self.total_cost = self.single_side_cost * 4 + transfer_fee * 2
    
    def calculate_opportunity(
        self,
        rate_high: float,
        rate_low: float,
        exchange_high: str,
        exchange_low: str
    ) -> dict:
        """
        计算套利机会
        
        策略：在费率高的交易所做空，在费率低的交易所做多
        """
        rate_diff = rate_high - rate_low
        net_profit_per_period = rate_diff - self.total_cost * 100
        
        return {
            'exchange_short': exchange_high,
            'exchange_long': exchange_low,
            'rate_high': rate_high,
            'rate_low': rate_low,
            'rate_diff': rate_diff,
            'total_cost': self.total_cost * 100,
            'net_profit_per_period': net_profit_per_period,
            'annual_return': net_profit_per_period * 3 * 365,
            'is_profitable': net_profit_per_period > 0
        }
    
    def find_opportunities(self, df_rates: pd.DataFrame, min_annual_return: float = 10) -> pd.DataFrame:
        """
        扫描套利机会
        """
        opportunities = []
        exchange_cols = [c for c in df_rates.columns if c not in ['symbol'] and not c.endswith('_diff')]
        
        for _, row in df_rates.iterrows():
            symbol = row['symbol']
            rates = {col: row[col] for col in exchange_cols if pd.notna(row[col])}
            
            if len(rates) < 2:
                continue
            
            # 找最高和最低费率
            max_exchange = max(rates, key=rates.get)
            min_exchange = min(rates, key=rates.get)
            
            opp = self.calculate_opportunity(
                rates[max_exchange],
                rates[min_exchange],
                max_exchange,
                min_exchange
            )
            opp['symbol'] = symbol
            
            if opp['annual_return'] >= min_annual_return:
                opportunities.append(opp)
        
        return pd.DataFrame(opportunities)

# 创建套利计算器
arb = CrossExchangeArbitrage()
arb_calc = arb
print(f'✅ 套利计算器初始化完成')
print(f'   单边成本: {arb.single_side_cost*100:.3f}%')
print(f'   总成本: {arb.total_cost*100:.3f}%')

In [ ]:
# 5.2 扫描套利机会

df_opportunities = arb.find_opportunities(df_rates, min_annual_return=5)

if not df_opportunities.empty:
    print('\n📊 发现套利机会：')
    print('=' * 80)
    for _, opp in df_opportunities.iterrows():
        print(f"\n{opp['symbol']}:")
        print(f"  做空: {opp['exchange_short']} ({opp['rate_high']:.4f}%)")
        print(f"  做多: {opp['exchange_long']} ({opp['rate_low']:.4f}%)")
        print(f"  费率差: {opp['rate_diff']:.4f}%")
        print(f"  净收益/期: {opp['net_profit_per_period']:.4f}%")
        print(f"  年化收益: {opp['annual_return']:.2f}%")
else:
    print('❌ 未发现符合条件的套利机会')

## 6. 成本分析

In [ ]:
# 6.1 详细成本分析

print('=' * 60)
print('📊 跨交易所套利成本分析')
print('=' * 60)

costs = {
    '交易所 A 开仓手续费': 0.04,
    '交易所 B 开仓手续费': 0.04,
    '交易所 A 平仓手续费': 0.04,
    '交易所 B 平仓手续费': 0.04,
    '滑点成本（估计）': 0.04,
    '资金转账费用': 0.02,
}

total = sum(costs.values())

for name, cost in costs.items():
    print(f'{name:<25} {cost:.2f}%')
print('-' * 60)
print(f'{"总成本":<25} {total:.2f}%')
print('=' * 60)

print(f'\n💡 盈亏平衡费率差: {total:.2f}%')
print(f'   年化盈亏平衡: {total * 3 * 365:.2f}%')

## 7. 实时机会扫描器

In [ ]:
# 7.1 实时扫描器

class ArbitrageScanner:
    """
    实时套利机会扫描器
    """
    def __init__(self, exchanges: dict, calculator: CrossExchangeArbitrage):
        self.exchanges = exchanges
        self.calculator = calculator
        self.history = []
    
    def scan(self, symbols: list, min_annual_return: float = 10) -> pd.DataFrame:
        """
        扫描当前套利机会
        """
        df_rates = compare_funding_rates(self.exchanges, symbols)
        opportunities = self.calculator.find_opportunities(df_rates, min_annual_return)
        
        if not opportunities.empty:
            opportunities['timestamp'] = datetime.now()
            self.history.append(opportunities)
        
        return opportunities
    
    def get_best_opportunity(self, symbols: list) -> dict:
        """
        获取最佳套利机会
        """
        opportunities = self.scan(symbols, min_annual_return=0)
        
        if opportunities.empty:
            return None
        
        best = opportunities.loc[opportunities['annual_return'].idxmax()]
        return best.to_dict()

# 创建扫描器
scanner = ArbitrageScanner(exchanges, arb)

# 扫描
print('📊 扫描套利机会...')
best = scanner.get_best_opportunity(SYMBOLS)

if best:
    print(f"\n🏆 最佳套利机会: {best['symbol']}")
    print(f"   策略: {best['exchange_short']} 做空 / {best['exchange_long']} 做多")
    print(f"   年化收益: {best['annual_return']:.2f}%")
else:
    print('❌ 未找到套利机会')

---

# 第三部分：基差跨所套利

---

## 8. 跨交易所基差分析

In [ ]:
# 8.1 获取各交易所价格

def get_prices(exchanges: dict, symbol: str = 'BTC/USDT:USDT') -> dict:
    """
    获取各交易所价格
    """
    results = {}
    
    symbol_map = {
        'binance': symbol,
        'okx': symbol,
        'bybit': symbol.replace('/USDT:USDT', 'USDT'),
    }
    
    for name, exchange in exchanges.items():
        try:
            sym = symbol_map.get(name, symbol)
            if sym in exchange.markets:
                ticker = exchange.fetch_ticker(sym)
                results[name] = {
                    'bid': ticker['bid'],
                    'ask': ticker['ask'],
                    'last': ticker['last'],
                    'spread': (ticker['ask'] - ticker['bid']) / ticker['bid'] * 100
                }
        except Exception as e:
            pass
    
    return results

# 获取 BTC 价格
print('📊 获取各交易所 BTC 价格...\n')
btc_prices = get_prices(exchanges)

print(f'{"交易所":<12} {"买一":<12} {"卖一":<12} {"价差":<10}')
print('-' * 50)
for name, data in btc_prices.items():
    print(f"{name:<12} {data['bid']:<12.2f} {data['ask']:<12.2f} {data['spread']:.4f}%")

In [ ]:
# 8.2 计算跨所基差

def calculate_cross_exchange_basis(prices: dict) -> pd.DataFrame:
    """
    计算跨交易所基差
    """
    exchanges = list(prices.keys())
    data = []
    
    for i, ex1 in enumerate(exchanges):
        for ex2 in exchanges[i+1:]:
            # ex1 买入, ex2 卖出
            buy_price = prices[ex1]['ask']
            sell_price = prices[ex2]['bid']
            basis_1 = (sell_price - buy_price) / buy_price * 100
            
            # ex2 买入, ex1 卖出
            buy_price_2 = prices[ex2]['ask']
            sell_price_2 = prices[ex1]['bid']
            basis_2 = (sell_price_2 - buy_price_2) / buy_price_2 * 100
            
            data.append({
                'pair': f'{ex1} → {ex2}',
                'buy_at': ex1,
                'sell_at': ex2,
                'basis': basis_1
            })
            data.append({
                'pair': f'{ex2} → {ex1}',
                'buy_at': ex2,
                'sell_at': ex1,
                'basis': basis_2
            })
    
    return pd.DataFrame(data)

df_basis = calculate_cross_exchange_basis(btc_prices)
print('\n📊 跨交易所基差：')
print(df_basis.to_string(index=False))

## 9. 基差套利策略实现

In [ ]:
# 9.1 基差套利机会检测

def find_basis_arbitrage(prices: dict, min_profit: float = 0.05) -> list:
    """
    寻找基差套利机会
    
    min_profit: 最小利润率 (%)
    """
    exchanges = list(prices.keys())
    opportunities = []
    
    # 估计成本
    trading_fee = 0.04 * 2  # 两边交易手续费
    slippage = 0.02  # 滑点
    total_cost = trading_fee + slippage
    
    for i, ex1 in enumerate(exchanges):
        for ex2 in exchanges[i+1:]:
            # ex1 买入, ex2 卖出
            buy_price = prices[ex1]['ask']
            sell_price = prices[ex2]['bid']
            gross_profit = (sell_price - buy_price) / buy_price * 100
            net_profit = gross_profit - total_cost
            
            if net_profit >= min_profit:
                opportunities.append({
                    'buy_exchange': ex1,
                    'sell_exchange': ex2,
                    'buy_price': buy_price,
                    'sell_price': sell_price,
                    'gross_profit': gross_profit,
                    'net_profit': net_profit
                })
            
            # 反向
            buy_price_2 = prices[ex2]['ask']
            sell_price_2 = prices[ex1]['bid']
            gross_profit_2 = (sell_price_2 - buy_price_2) / buy_price_2 * 100
            net_profit_2 = gross_profit_2 - total_cost
            
            if net_profit_2 >= min_profit:
                opportunities.append({
                    'buy_exchange': ex2,
                    'sell_exchange': ex1,
                    'buy_price': buy_price_2,
                    'sell_price': sell_price_2,
                    'gross_profit': gross_profit_2,
                    'net_profit': net_profit_2
                })
    
    return opportunities

# 检测基差套利机会
arb_ops = find_basis_arbitrage(btc_prices, min_profit=0.01)

if arb_ops:
    print('📊 发现基差套利机会：')
    for op in arb_ops:
        print(f"  {op['buy_exchange']} 买入 → {op['sell_exchange']} 卖出")
        print(f"  净利润: {op['net_profit']:.4f}%")
else:
    print('❌ 未发现基差套利机会（价差不足以覆盖成本）')

---

# 第四部分：实战系统

---

## 10. 多交易所 API 连接

In [ ]:
# 10.1 交易所管理器

class ExchangeManager:
    """
    多交易所连接管理器
    """
    def __init__(self):
        self.exchanges = {}
        self.status = {}
    
    def add_exchange(self, name: str, config: dict):
        """
        添加交易所
        """
        exchange_class = getattr(ccxt, name)
        self.exchanges[name] = exchange_class(config)
        self.status[name] = 'initialized'
    
    def connect_all(self):
        """
        连接所有交易所
        """
        for name, exchange in self.exchanges.items():
            try:
                exchange.load_markets()
                self.status[name] = 'connected'
                print(f'✅ {name}: 已连接')
            except Exception as e:
                self.status[name] = f'error: {e}'
                print(f'❌ {name}: {e}')
    
    def get_status(self) -> dict:
        return self.status
    
    def fetch_all_funding_rates(self, symbol: str) -> dict:
        """
        获取所有交易所的资金费率
        """
        results = {}
        for name, exchange in self.exchanges.items():
            if self.status[name] == 'connected':
                try:
                    rate = exchange.fetch_funding_rate(symbol)
                    results[name] = rate['fundingRate'] * 100
                except:
                    pass
        return results

# 示例使用
print('📊 交易所管理器示例：')
print('   manager = ExchangeManager()')
print('   manager.add_exchange("binance", {...})')
print('   manager.connect_all()')

## 11. 套利信号监控

In [ ]:
# 11.1 信号监控系统

class ArbitrageMonitor:
    """
    套利信号监控系统
    """
    def __init__(self, exchanges: dict, symbols: list):
        self.exchanges = exchanges
        self.symbols = symbols
        self.alerts = []
        self.calculator = CrossExchangeArbitrage()
    
    def check_signals(self, min_annual_return: float = 20) -> list:
        """
        检查套利信号
        """
        signals = []
        df_rates = compare_funding_rates(self.exchanges, self.symbols)
        opportunities = self.calculator.find_opportunities(df_rates, min_annual_return)
        
        for _, opp in opportunities.iterrows():
            signal = {
                'timestamp': datetime.now(),
                'symbol': opp['symbol'],
                'action': f"SHORT@{opp['exchange_short']} / LONG@{opp['exchange_long']}",
                'annual_return': opp['annual_return'],
                'rate_diff': opp['rate_diff']
            }
            signals.append(signal)
            self.alerts.append(signal)
        
        return signals
    
    def print_status(self):
        """
        打印监控状态
        """
        print('\n' + '=' * 60)
        print(f'📊 套利监控状态 - {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
        print('=' * 60)
        print(f'监控币种: {len(self.symbols)}')
        print(f'历史信号: {len(self.alerts)}')
        
        if self.alerts:
            print('\n最近信号:')
            for alert in self.alerts[-5:]:
                print(f"  [{alert['timestamp'].strftime('%H:%M')}] {alert['symbol']}: {alert['action']} ({alert['annual_return']:.1f}% 年化)")

# 创建监控器
monitor = ArbitrageMonitor(exchanges, SYMBOLS)
monitor.check_signals(min_annual_return=10)
monitor.print_status()

## 12. 风险管理

### 12.1 风险管理框架

```
┌─────────────────────────────────────────────────────────────────┐
│                      风险管理框架                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 仓位管理                                                   │
│     ├── 单币种最大仓位: 总资金的 20%                           │
│     ├── 单交易所最大仓位: 总资金的 40%                         │
│     └── 总杠杆控制: 不超过 3x                                  │
│                                                                 │
│  2. 风险控制                                                   │
│     ├── 费率反转止损: 费率差 < 0 时平仓                        │
│     ├── 最大回撤限制: 5%                                       │
│     └── 单笔最大亏损: 1%                                       │
│                                                                 │
│  3. 执行控制                                                   │
│     ├── 滑点保护: 预期滑点 > 0.1% 时暂停                       │
│     ├── 延迟监控: API 延迟 > 500ms 时暂停                      │
│     └── 流动性检查: 深度不足时减小仓位                         │
│                                                                 │
│  4. 系统风控                                                   │
│     ├── 交易所 API 异常监控                                    │
│     ├── 资金安全分散                                           │
│     └── 紧急平仓机制                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 12.2 风险管理器

class RiskManager:
    """
    风险管理器
    """
    def __init__(
        self,
        max_position_per_symbol: float = 0.2,
        max_position_per_exchange: float = 0.4,
        max_leverage: float = 3.0,
        max_drawdown: float = 0.05,
        min_rate_diff: float = 0.02
    ):
        self.max_position_per_symbol = max_position_per_symbol
        self.max_position_per_exchange = max_position_per_exchange
        self.max_leverage = max_leverage
        self.max_drawdown = max_drawdown
        self.min_rate_diff = min_rate_diff
    
    def check_position_limit(self, current_positions: dict, symbol: str, amount: float, total_capital: float) -> bool:
        """
        检查仓位限制
        """
        symbol_exposure = current_positions.get(symbol, 0) + amount
        if symbol_exposure > total_capital * self.max_position_per_symbol:
            return False
        return True
    
    def check_rate_diff(self, rate_high: float, rate_low: float) -> bool:
        """
        检查费率差是否足够
        """
        return (rate_high - rate_low) >= self.min_rate_diff
    
    def should_close_position(self, rate_high: float, rate_low: float) -> bool:
        """
        判断是否应该平仓
        """
        return (rate_high - rate_low) < 0

risk_mgr = RiskManager()
print('✅ 风险管理器已初始化')
print(f'   单币种最大仓位: {risk_mgr.max_position_per_symbol*100}%')
print(f'   最大杠杆: {risk_mgr.max_leverage}x')
print(f'   最小费率差: {risk_mgr.min_rate_diff}%')

## 13. 回测与绩效分析

In [ ]:
# 13.1 简化回测框架

def backtest_cross_exchange_arbitrage(
    rate_diffs: np.ndarray,
    initial_capital: float = 10000,
    total_cost: float = 0.002,  # 0.2%
    min_rate_diff: float = 0.0003
) -> dict:
    """
    回测跨交易所套利策略
    
    参数:
        rate_diffs: 费率差序列（两个交易所费率之差）
        initial_capital: 初始资金
        total_cost: 总成本
        min_rate_diff: 最小费率差
    """
    capital = initial_capital
    equity_curve = [initial_capital]
    
    position = 0  # 0: 无仓位, 1: 有仓位
    trades = 0
    total_pnl = 0
    
    for rate_diff in rate_diffs:
        if position == 0 and rate_diff >= min_rate_diff:
            # 开仓
            capital -= capital * total_cost  # 扣除开仓成本
            position = 1
            trades += 1
        elif position == 1:
            # 持仓收益
            pnl = capital * rate_diff
            capital += pnl
            total_pnl += pnl
            
            # 检查平仓条件
            if rate_diff < 0:
                capital -= capital * total_cost  # 扣除平仓成本
                position = 0
        
        equity_curve.append(capital)
    
    # 计算指标
    total_return = (capital - initial_capital) / initial_capital
    
    return {
        'initial_capital': initial_capital,
        'final_capital': capital,
        'total_return': total_return * 100,
        'trades': trades,
        'equity_curve': equity_curve
    }

# 模拟数据回测
np.random.seed(42)
simulated_rate_diffs = np.random.normal(0.0003, 0.0002, 1000)  # 模拟1000期费率差

result = backtest_cross_exchange_arbitrage(simulated_rate_diffs)

print('📊 回测结果（模拟数据）')
print('=' * 40)
print(f'初始资金: ${result["initial_capital"]:,.2f}')
print(f'最终资金: ${result["final_capital"]:,.2f}')
print(f'总收益: {result["total_return"]:.2f}%')
print(f'交易次数: {result["trades"]}')

In [ ]:
# 13.2 回测可视化

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=result['equity_curve'],
    mode='lines',
    name='账户权益',
    line=dict(color='blue', width=2)
))
fig.add_hline(y=result['initial_capital'], line_dash='dash', line_color='gray')
fig.update_layout(
    title=f'跨交易所套利回测 - 总收益: {result["total_return"]:.2f}%',
    xaxis_title='期数',
    yaxis_title='账户权益 (USDT)',
    height=400
)
fig.show()

---

## 总结

### 核心要点

1. **跨交易所套利** 利用不同交易所之间的费率差或价格差获利

2. **主要套利类型**：
   - 资金费率套利（最常见）
   - 基差套利
   - 三角套利

3. **成本控制** 是盈利关键：
   - 选择低手续费交易所
   - 使用 Maker 单
   - 控制滑点

4. **风险管理**：
   - 仓位限制
   - 费率反转止损
   - 交易所分散

### 实战建议

- 从小资金开始测试
- 重点关注高流动性币种
- 建立完善的监控系统
- 定期优化策略参数

### 下一步学习

- [ ] 高频交易与订单簿分析
- [ ] 期权与永续合约组合策略